## **1. Import Necessary libraries**

In [51]:
import cv2
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

## **2. Load Images and Extract Features**

### **Function to Extract HOG Features**

In [52]:
def extract_hog_features(image):
    hog_descriptor=cv2.HOGDescriptor()
    hog_features=hog_descriptor.compute(image)
    return hog_features

### **Loading Images and Labels**

In [53]:
# Load and prepare the dataset

def load_images_and_labels(folder):
    feature_vectors = []
    labels = []

    # For each folder
    for label, folder_name in enumerate(folder):
        folder_path = os.path.join('D:\SEM 8\Computer Vision\Animals', folder_name)
        image_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.jpg')]

        for image_path in image_paths:
            image = cv2.imread(image_path)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            resized_image = cv2.resize(gray_image, (64, 128))

            features = extract_hog_features(resized_image)
            feature_vectors.append(features)
            labels.append(label)

    return np.array(feature_vectors), np.array(labels)

In [54]:
folder=['cats', 'dogs', 'snakes']
feature_vectors,labels=load_images_and_labels(folder)

In [55]:
feature_vectors

array([[0.2671464 , 0.06417033, 0.02682497, ..., 0.24651693, 0.24651693,
        0.23092984],
       [0.18998984, 0.16621618, 0.08479402, ..., 0.2158035 , 0.25118038,
        0.17661369],
       [0.0990245 , 0.07832853, 0.13829508, ..., 0.0590654 , 0.05840059,
        0.07236026],
       ...,
       [0.18563877, 0.1776225 , 0.14491385, ..., 0.13561237, 0.04299479,
        0.07314017],
       [0.09971177, 0.18428993, 0.13012731, ..., 0.22553225, 0.22553225,
        0.19224657],
       [0.0013138 , 0.00162204, 0.00268776, ..., 0.20706685, 0.08148804,
        0.02074043]], dtype=float32)

In [56]:
labels

array([0, 0, 0, ..., 2, 2, 2])

## **3. Splitting the dataset**

In [57]:
x_train,x_test,y_train,y_test=train_test_split(feature_vectors,labels,test_size=0.3,random_state=42)

## **4. Creating & Training the SVM Classifier**

In [58]:
svm_classifier = cv2.ml.SVM_create()
svm_classifier.setKernel(cv2.ml.SVM_LINEAR)
svm_classifier.setType(cv2.ml.SVM_C_SVC)
svm_classifier.setC(1)

# Train the SVM Classifier
svm_classifier.train(x_train, cv2.ml.ROW_SAMPLE, y_train)


True

## **5. Saving the Trained model**

In [59]:
svm_classifier.save('svm-animal.xml')

## **6. Evaluate the model**

In [60]:
test_result, test_predictions=svm_classifier.predict(x_test)

# Calculate accuracy
test_accuracy=np.mean(test_predictions == y_test)

print(f'Test Accuracy: {test_accuracy*100:.2f}')



Test Accuracy: 33.45


## **7. Test on new images**

In [65]:
def classify_new_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Image at {image_path} could not be loaded.")
        return
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray_image, (64, 128))
    hog_features = extract_hog_features(resized_image)

    # Reshape the feature to 2D array
    feature_reshaped = hog_features.reshape(1, -1).astype(np.float32)

    # SVM Classifier
    svm_classifier = cv2.ml.SVM_load('svm-animal.xml')

    # Make a prediction
    prediction_res, predicted_res = svm_classifier.predict(feature_reshaped)

    # Get the label
    predicted_label = int(predicted_res[0][0])

    # Output
    if predicted_label == 0:
        print("Cat")
    elif predicted_label == 1:
        print("Dog")
    elif predicted_label == 2:
        print("Snake")
    else:
        print("Unknown")

classify_new_image(r'D:\SEM 8\Computer Vision\Animals\cats\0_0998.jpg')

Cat
